# Lab 07 - Data Wrangling Skills Part II
Two main concepts are covered in this lab: 
1. data enrichment; and
2. feature engineering. 
Some of the skills that you have learned are needed to complete the exercises. 

## 7a.0 Data Enrichment
 In this section, we will use two data sets. One data set is a fake data and another one is the air quality data from UCI (the data set which has be used in lab06 under section Date Time).

1.	Create a fake data with movies title and two users’ profile. The rating with value `1` indicates extremely dislike whereas `10` indicates extremely like.  

In [ ]:
import pandas as pd
# create data. In real world, this data may be from database
movies = {'movie_id': [1000, 1001, 1002, 1003],
          'title': ['Toy Story', 'Jumanji', 'Iron Man', 'The Secrets of Dumbledore'],
          'genre': ['Adventure|Animation|Children', 'Adventure|Children|Fantasy', 'Action|Adventure|Sci-Fi', 'Adventure|Fantasy|Family']}

users = {'user_id': [100, 100, 101, 101, 102, 102, 103,103, 103, 104, 104, 105, 
                     106, 106, 106, 107, 107, 108, 108, 109],
         'DOB': [20000221, 20000221, 19901212, 19901212, 19831125, 19831125, 
                 19880920, 19880920, 19880920, 19750614,19750614, 19600522, 19950505, 
                 19950505, 19950505, 19770723, 19770723, 20010101, 20010101, 19990608],
         'movie_id': [1000, 1001, 1000, 1002, 1001,1002, 1000, 1002, 1002, 1001, 1002, 
                      1001, 1000, 1001, 1002, 1001, 1002, 1000, 1001, 1002],
         'timestamp': [20210728, 20210727, 20210726, 20210728, 20210727, 20210726, 20210728, 
                       20210727, 20210726,20210728, 20210727, 20210726, 20210728, 20210727, 20210726,
                       20210728, 20210727, 20210726, 20210727, 20210726],
         'rating': [9,8,7,5,8,7,9,10,5,6,8,8, 9, 8, 6, 8, 7, 8,6,8]}

users1 = {'user_id': [110, 110, 111, 112, 112, 112, 113,113, 113, 114, 114, 115],
         'DOB': [20020202, 20020202, 19901212, 19831130, 19831130, 19831130, 19880905, 
                 19880905, 19880905, 19750610,19750610, 19600512],
         'movie_id': [1000, 1001, 1000, 1002, 1001,1002, 1000, 1002, 1002, 1001, 1002, 1001],
         'timestamp': [20210728, 20210727, 20210726, 20210728, 20210727, 
                       20210726, 20210728, 20210727, 20210726,20210728, 20210727, 20210726],
         'rating': [8,8,9,6,5,8,9,10,8,7,5,8]}


### Exercise 1: Data Frame and Histogram

This exercise is to let you practice your python skill in creating data frame and observe the data through histogram.\
Perform the following:

Ex 1.1 : Now, save both movies, users and users1 into three different data frames named as `Data1`, `Data2` and `Data3` respectively.\
\
Ex 1.2 : Display the first few rows of each data frame. \
\
Ex 1.3 : Explore the feature “rating” by using histogram and set the number of bins = 5. [Hints: the function is ``.hist(bins=X)``]


2.	The genre in Data1 is a string, such as “`Advanture|Animation|Children`”. Change each of the word into a list by manipulating the string.

In [ ]:
# split the genre column strings at '|' to make lists
Data1.genre = Data1.genre.str.split('|')
display(Data1)

3.	Transform the DOB in Data2 and Data3 from string to date.

In [ ]:
Data2['DOB']=Data2['DOB'].apply(lambda x: pd.to_datetime(str(x), format='%Y-%m-%d'))
Data3['DOB']=Data3['DOB'].apply(lambda x: pd.to_datetime(str(x), format='%Y-%m-%d'))

### Exercise 2: Adding New Columns

Ex2.1.	Based on the DOB, count the age of each user in Data2 and Data3. [Hints: to_datetime(‘today’) will get you the date of today but you need only the year]\
\
Ex2.2.	Add the information in Ex2.1 as a new column to Data2 and Data3 respectively. Then, display first few rows of both data


## 8a.1 Simple Binning

1.	Obtain `minAge` and `maxAge` from `Data2` through column `Age`.

In [ ]:
import numpy as np
minAge = Data2['Age'].min()
maxAge = Data2['Age'].max()

2. Set the linspace (linearly spaced values) between `minAge` and `maxAge` to 4 intervals. With 4 intervals, we can set 3 categories. 

In [ ]:
bins=np.linspace(minAge,maxAge,4)
labels=['young','middle age','senior']

3.  Use a function called `cut` to have equal width binning. 

In [ ]:
Data2['bins'] =pd.cut(Data2['Age'], bins=bins, labels=labels, include_lowest=True)
Data2

4.	Use `qcut` and observe the equal frequency binning (approximate). Discuss the differences of output after apply `cut` and `qcut`.

In [ ]:
Data2['bin_qcut'] = pd.qcut(Data2['Age'], q=3, labels=labels, precision=1)
display(Data2)

## 7a.2 Clustering and Binning
Clustering techniques can be used to split the data and are one of the binning techniques. 

#### A. In this section, we use k-mean.

1. Import k-mean from sklearn.

In [ ]:
from sklearn.cluster import KMeans

2. Create a variable `Data4` to store the `Age` column information in `Data2`.

In [ ]:

Data4 = Data2[['Age']]

3.	Set the number of clusters in k-mean to 3 and train on `Data4`.

In [ ]:
kmeans = KMeans(n_clusters=3)
kmeans.fit(Data4)

4.	Add the cluster result from k-mean as an additional columns and name the column as “cluster”. 

In [ ]:
Data4.loc[:, 'cluster'] =kmeans.labels_

5.	Name the labels by setting 0 as Young, 1 as Middle Age and 2 as Senior.

In [ ]:
labels = {0: "Young", 1: "Middle Age", 2: "Senior"}

6.	Apply the labels to `Data4['cluster']` and add it into Data2 as "`class_kmeans`"

In [ ]:
Data2["class_kmeans"] = Data4["cluster"].apply(lambda x: labels[x])

7.	Compare all the other binning results and observe the differences.

In [ ]:
print(Data2)

#### B.	Another simple clustering technique is Fisher-Jenks algorithm (natural break). 
The example is demonstrated here. 

1.	Install jenkspy if you have not done so (Batch file, or executed in windows)

In [ ]:
pip install jenkspy

2.	Import the module and set the break of classes to 3.

In [ ]:
import jenkspy
Breaks_nature = jenkspy.jenks_breaks(Data2['Age'], nb_class=3)

3.	Use function `cut` to break the data according to the break information in `Breaks_nature` and store the information in a new column `bincut_break`. Show the data of this new column by using histogram.

In [ ]:
Data2['bincut_break'] = pd.cut(Data2['Age'], bins=Breaks_nature, labels=labels, include_lowest=True)
Data2['bincut_break'].hist()

4.	Display `Data2`. 

In [ ]:
display(Data2)

## 7a.3 Concatenate, Merging and Joining

We can join data frames together to check the intersection/overlapping records between two data sets or combine related information together into one data frame and form a larger data set.

1.	Apply a simple concatenate function to combine Data1 and Data2 by columns. Print screen your result and paste in a word document. 

In [ ]:
con1 = pd.concat([Data1, Data2], axis=1)
display(con1) 

2.	Add in the parameter join=’inner’. Observe the difference between the output with join and the output in No. 1. 

In [ ]:
con1 = pd.concat([Data1, Data2], axis=1, join='inner')
display(con1)

3.	The following command is applying merge to combine Data1 and Data2. 

In [ ]:
outer_Merge = pd.merge(Data1, Data2, how="outer",on='movie_id', suffixes=("_x","_y"))
display(outer_Merge)

4.	Use join. Print screen the result. 

5.	Observe and dicuss the differences in the result if any.

### Exercise 3: Combining the Data
Ex3.1.	Concatenate Data1 and Data2 by rows.

Ex3.2.	Concatenate Data2 and Data3 by columns.

Ex3.3.	Concatenate Data2 and Data3 by rows.

Ex3.4.	Use join, and change the parameter of how to right, outer and inner.

Ex3.5.	Observe and discuss the differences among the concatenate results and the join results.


## 7a.4 Summarizing/Aggregating

Summary and aggregation function in Pandas are useful in producing meaningful results. \
Here are some examples:
1.	To see the minimum and maximum rating per user and group according to age group \
(based on the result from function cut in `bins`).


In [ ]:
Data2.groupby('user_id').agg({'rating':['min','max']})
Data2.groupby('bins').agg({'rating':['min','max']})

### Exercise 4: Groupby
Ex4.1.	Check the min and max rating per age group based on the result from qcut (`bin_qcut`).

Ex4.2.	Check the min and max rating per age group based on the result from k-means (`class_kmeans`).

Ex4.3.	Display the average rating of each movie.

Ex4.4.	Display the min and max age of user for each movie.

Ex4.5.	Count the number of users for each movie.


## 7a.5 Resampling

In this section, the air quality data is used. You can reuse the command where we have converted date time into the proper data type.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

data = pd.read_csv(r"C:\Users\XXX\AirQualityUCI.csv", header=[0], sep=";")
#this portion handle the missing data
threshold=data.isnull().sum()/ len(data) 
#print(threshold)
remove_col = threshold[threshold>0.8].index #get the index of each columns and store in remove_col
data.drop(remove_col, axis=1, inplace = True)
#print(data.columns)
threshold = data.isnull().sum(axis=1)/(len(data.columns)-1)
#print(threshold)
remove_row = threshold[threshold>0.8].index
data.drop(remove_row, axis=0, inplace = True)
print(data.head()) #check to make sure it moves only those with missing values >80%
print(len(data))
data['Date'] = pd.to_datetime(data['Date'])
data['Time'] = pd.to_datetime(data['Time'], format = '%H.%M.%S').dt.time

The features of this data are as below:

0. Date (DD/MM/YYYY) 
1. Time (HH.MM.SS) 
2. True hourly averaged concentration CO in mg/m^3 (reference analyzer) 
3. PT08.S1 (tin oxide) hourly averaged sensor response (nominally CO targeted) 
4. True hourly averaged overall Non Metanic HydroCarbons concentration in microg/m^3 (reference analyzer) 
5. True hourly averaged Benzene concentration in microg/m^3 (reference analyzer) 
6. PT08.S2 (titania) hourly averaged sensor response (nominally NMHC targeted) 
7. True hourly averaged NOx concentration in ppb (reference analyzer) 
8. PT08.S3 (tungsten oxide) hourly averaged sensor response (nominally NOx targeted) 
9. True hourly averaged NO2 concentration in microg/m^3 (reference analyzer) 
10. PT08.S4 (tungsten oxide) hourly averaged sensor response (nominally NO2 targeted) 
11. PT08.S5 (indium oxide) hourly averaged sensor response (nominally O3 targeted) 
12. Temperature in Â°C 
13. Relative Humidity (%) 
14. AH Absolute Humidity

1.	Set the date as index and display the first few rows of the data. 

In [ ]:
data1=data.set_index(pd.DatetimeIndex(data['Date'])).drop("Date", axis=1)
data1.head()

2.	Resample the data by producing average value by week. This is called downsample.

In [ ]:
data1.resample("W").mean().ffill()

3.	Produce the min and max value by week. Note that there are some values are `NaN`.

In [ ]:
data1.resample("W").agg(['min','max'])

4.	Upsampling: Produce the average value by 30 minutes and forward fill the values

In [ ]:
data1.resample("30min").mean().ffill()

## 7b: Feature Selection

In this section, use the data from https://archive.ics.uci.edu/ml/datasets/Real+estate+valuation+data+set

Here is the information about the data:\
The market historical data set of real estate valuation are collected from Sindian Dist., New Taipei City, Taiwan. The real estate valuation is a regression problem. The data set was randomly split into the training data set (2/3 samples) and the testing data set (1/3 samples).

Attribute Information:

The inputs are as follows:\
X1=the transaction date (for example, 2013.250=2013 March, 2013.500=2013 June, etc.) \
X2=the house age (unit: year) \
X3=the distance to the nearest MRT station (unit: meter) \
X4=the number of convenience stores in the living circle on foot (integer) \
X5=the geographic coordinate, latitude. (unit: degree) \
X6=the geographic coordinate, longitude. (unit: degree) \
The output is as follow: \
Y= house price of unit area (10000 New Taiwan Dollar/Ping, where Ping is a local unit, 1 Ping = 3.3 meter squared)


## 7b.1: Correlation and heatmap

1.	Import all the modules and data

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
np.random.seed(123)
data = pd.read_excel('Real estate valuation data set.xlsx')
print(data.head())

2.	Drop the first columns which contains only numbers.

In [ ]:
data = data.iloc[:,1:8]
print(data.head())

3.	Produce correlation matrix

In [ ]:
cor = data.corr()
cor

4.	Produce the heat map. \
Based on 3 and 4,\
a.	Identify the positive and negative correlated variables. \
b.	Identify variables that have very low correlation.


In [ ]:
plt.figure(figsize=(10,6))
sns.heatmap(cor, annot=True)

5.	The dependent variable is Y, thus we only interest to know which variables are meaningful to determine Y. Let us set the correlation threshold as 0.5. When the features are having correlation +-0.5 and above, then the features are selected.  

In [ ]:
threshold = 0.5
s = abs(cor['Y house price of unit area'])
selectedFeature = s[s>0.5]
print(selectedFeature)

## 7b.2 Variance Inflation Factor (VIF)

1.	Import VIF

In [ ]:
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

2.	Declare x (independent variables) and y (dependent variable)

In [ ]:
x = data.iloc[:,0:6] #0 to 5, need to set as 0:6
y = data.iloc[:, 6]
print(x.head())
print(y.head())

3.	Run VIF directly and observe the result.

In [ ]:
vif=[variance_inflation_factor(x.values, i) for i in range(x.shape[1])]
print(vif)

4.	Then, try to run multiple regression and generate the VIF. Now, observe the result.\
<u>Which features should be selected?</u>

In [ ]:
#run regression and generate VIF
Reg = sm.OLS(y, x).fit() #OLS = ordinary linear squares regression model
pd.DataFrame({'variables': x.columns[0:6], 'VIF':[variance_inflation_factor(x.values, i) for i in range(len(x.columns[0:6]))]})

'''Feature 1, 2, 3, 4 and 6. 5 considers too high compare to the other features. '''

## 7b.3 Wrapper Method – Forward Selection

1.	Import mlxtend library. \
(mlxtend = machine learning extension, this library contains a lot of interesting tools for machine learning tasks and data analysis.)

In [ ]:
conda install -c conda-forge mlxtend

2.	Import modules

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score as acc
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
from sklearn.linear_model import LinearRegression

3.	Build linear regression model to use in feature selection

In [ ]:
LR = LinearRegression()
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size = 0.3, random_state = 0)

4.	Build step forward feature selection

In [ ]:
step_forward = sfs(LR, k_features = 4, forward = True, floating = False, scoring = 'r2', verbose = 2, cv = 5)

5.	Perform step forward feature selection

In [ ]:
step_forward = step_forward.fit(X_train, Y_train)
print(list(step_forward.k_feature_idx_))

6.	In this command, we use random forest to select useful features. Which features are selected?

In [ ]:
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import roc_auc_score
from mlxtend.feature_selection import SequentialFeatureSelector 

feature_selector = SequentialFeatureSelector(RandomForestClassifier(n_jobs=-1), k_features=4, forward=True,verbose=2, scoring='roc_auc', cv=5)
features=feature_selector.fit(X_train, Y_train)
print(list(features.k_feature_idx_))

## 7b.4 Wrapper Method – Backward Elimination

1.	Use linear regression to perform backward elimination. <u>Which features are selected?</u>

In [ ]:
cols=list(x.columns) #get the columns
while(len(cols)>0):  #when there are columns available in cols
    p=[]
    x_1=x[cols]
    x_1=sm.add_constant(x_1)
    model=sm.OLS(y,x_1).fit() 
    #sm is an object of stat models, and we use OLS froms statmodels library
    p=pd.Series(model.pvalues.values[1:], index=cols)
    pmax=max(p)
    features_with_p_max=p.idxmax()
    if(pmax>0.05):  #p is the null hypothesis p value, if greater than 0.05, remove feature
        cols.remove(features_with_p_max)
    else:
        break
summary = model.summary()
print(summary)
selected_features=cols
print(selected_features)

## 7b.5 Embedded Method

1.	Use random forest to perform backward elimination. <u>Which features are selected?</u>

In [ ]:
feature_selector = SequentialFeatureSelector(RandomForestClassifier(n_jobs=-1), k_features=4, forward=False,verbose=2, scoring='accuracy', cv=5)
features=feature_selector.fit(np.array(X_train.fillna(0)), Y_train)
print(list(features.k_feature_idx_))

2.	Use Lasso Regression method. <u>Which features are selected?</u>

In [ ]:
from sklearn.linear_model import LassoCV
reg = LassoCV()
reg.fit(x,y)
print("Best alpha using built-in LassoCV: %f" % reg.alpha_)
print("Best score using built-in LassoCV: %f" %reg.score(x,y))
coef=pd.Series(reg.coef_, index=x.columns)
print(coef)
print("Lasso picked " + str(sum(coef != 0))+"variables and eliminated the other " + str(sum(coef == 0)) + " variables")

#the plot figure below shows which features are selected
imp_coef = coef.sort_values()
import matplotlib
matplotlib.rcParams['figure.figsize'] = (8.0, 10.0)
imp_coef.plot(kind = "barh")
plt.title("Feature importance using Lasso Model")

### Exercise 5: Feature Selection
1.	Download breast cancer data set from this link: https://archive.ics.uci.edu/ml/datasets/breast+cancer+wisconsin+(diagnostic)
[Hints: you can refer to lab 6 to use a ready data set by impute the missing data with median]
2.	Perform feature selection as below: 

a.	applies a forward selection by using linear regression, set k_features = 4. Record the selected features. \
b.	applies a filter method by using correlation. Select features with correlation > 0.5. Record the selected features.\
c.	applies an embedded method by using Lasso Regression. Record the selected features.
